# Neutron Stars Integrator
This notebook solves the TOV equations for static neutron stars described by an EOS that includes a QCD and a vacuum energy phase transition. 

The units in which the notebook works are in G=c=M_sun=1, but rescaling factors are provided.

**Important note**: there is no need to create the 'modified' EOSs beforehand (e.g. through the eos_generator notebook provided in the repository). The 'modified' EOS are created within the code, starting from a standard EOS (sly and ap4 are provided), a dataset for segments speed of sound parametrization modelling the QCD phase (a smaller and larger dataset are provided) and a value of vacuum energy shift.

The general description for the EOS model is provided in **arXiv:23...**.

## Import

In [ ]:
using DifferentialEquations, DelimitedFiles, Interpolations, Symbolics, Printf, Plots, Roots, QuadGK

## Load data and units conventions

Note: the usual dataset for the AP4 and the SLy gave all quantities in g/cm^3
The dataset in Rescaledsly.dat and Rescaledap4.dat have been rescaled to G=c=M_Sun=1
The data for the (ρ,c_s) segments are also in G=c=M_Sun=1

Smaller dataset corresponds to matrixrhoS.dat and matrixcsS.dat

Larger dataset corresponds to matrixrhoL.dat and matrixcsL.dat

In [ ]:
#LOADING DATA
sly = readdlm("Rescaledsly.dat", Float64) #p,ϵ,ρ
ap4 = readdlm("Rescaledap4.dat", Float64); #p,ϵ,ρ
matrix_ρ = readdlm("matrixrhoS.dat", Float64) #loading ρ coordinates for segments, 100 EOS
matrix_cs = readdlm("matrixcsS.dat", Float64); #loading cs coordinates for segments, 100 EOS
#matrix_ρ = readdlm("matrixrhoL.dat", Float64) #loading ρ coordinates for segments, 1000 EOS
#matrix_cs = readdlm("matrixcsL.dat", Float64);#loading cs coordinates for segments, 1000 EOS

In [ ]:
#CONSTANTS FOR UNITS
const cspeed = 2.997 * 10.0^10 #in cm s^-1
const hbar = 6.582 * 10.0^-22 #in Mev s
const MeV = 1.602 * 10.0^-6 #in g cm^2 s^-1
const Kp = MeV/(hbar^3 * cspeed^3) #in g and cm
const Kρ = MeV/(hbar^3 * cspeed^5) #in g and cm
const fm = (10.0^-13)^-3 #fm^-3 in cm^-3
const n0 = 0.16 * fm #nuclear saturation density in cm^-3
const m = 1.675 * 10.0^-24 #neutron mass in g
const Msun = 1.988 * 10.0^33 #mass of the Sun in g
const Gconst = 6.67 * 10.0^-11 * 10.0^6/10.0^3 #in cm^3/g/s^2
const Lu = Msun^-1 * cspeed^2 * Gconst^-1 #1cm in G=c=Msun=1
const Pu = Msun^-1 / Lu^3 / cspeed^2  #transforming a pressure in g/cm/s^2 in G=c=Msun=1 units
const ρu = Msun^-1 / Lu^3; #transforming a mass density in g/cm^3 in G=c=Msun=1 units
#rescaling factor for λ to check with GW results
const KK = (2.997 * 10.0^5)^2/(6.67 * 10.0^-20 * 1.988 * 10.0^30);

## Functions definition

In [ ]:
#Function to find (closest) position in array of certain value
#!!It works for increasing values on column!!
function findpos(matrix,value,column)
    for b = 1:length(matrix[:,column])
        Δx1 = abs(matrix[b,column] - value)
        Δx2 = abs(matrix[b+1,column] - value)
        if matrix[b+1,column] >= value
            if Δx1 < Δx2
                return b
                break
            else
                return b+1
                break
            end
        end
    end
end;

#Function to find (closest) position in array of certain value
#!!It works for decreasing values on column!!
function findposDec(matrix,value,column)
    for j = 1:length(matrix[:,column])
        Δx1 = abs(matrix[j,column] - value)
        Δx2 = abs(matrix[j+1,column] - value)
        if matrix[j+1,column] < value
            if Δx1 < Δx2
                return j
                break
            else
                return j+1
                break
            end
        end
    end
end;

In [ ]:
#Evaluating derivative functions to find quantities at threshold
function derThr(eos, k)
    s = 0.
    for j = 1:k
        Δp = eos[j+1,1] - eos[j,1]
        Δϵ = eos[j+1,2] - eos[j,2]
        s = Δp/Δϵ
    end
    return s
end

#creating an interpolated function for speed of sound modification
function csInterp(eos, k, p1, matr_ρ, matr_cs, ds)
    #define matrix to store (ρ,cs) for segments modification
    segm_matrix = Matrix{Float64}(undef,0,2) 
    segm_matrix = vcat(segm_matrix, [eos[k,3] p1])
    for j = 1:7
      segm_matrix = vcat(segm_matrix, [matr_ρ[ds,j] matr_cs[ds,j]])  
    end
    return linear_interpolation(segm_matrix[:,1],segm_matrix[:,2])
end

#create high energy EOS
function heEOS(he_eos::AbstractMatrix{T}, eos, k, cs_interp) where T
    Δρ = 10.0^-5
    M = div((10.0^-3 - eos[k,3]),10.0^-5)+div((10.0^-2 - 10.0^-3),10.0^-4)+
        div((ρfin - 10.0^-2),10.0^-3)    
    ρold = eos[k,3]
    ϵold = eos[k,2]
    pold = eos[k,1]
    
    for j = 1:M-1
        ρnew = ρold + Δρ
        ϵnew = ϵold + Δρ * (ϵold + pold)/ρold
        pnew = pold + cs_interp(ρold)^2 * Δρ * (ϵold + pold)/ρold #cs^2=dp/dϵ
        if ρnew > ρfin #break cycle if we reach last mass density
            break
        end
        he_eos = vcat(he_eos, [pnew ϵnew ρnew])
        #store new data in old variable for next loop
        ρold = ρnew 
        ϵold = ϵnew
        pold = pnew
        if ρnew < 10.0^-3
            Δρ = 10.0^-5
        elseif ρnew < 10.0^-2
            Δρ = 10.0^-4
        elseif ρnew < 10^-1
            Δρ = 10.0^-3
        else 
            Δρ = 10.0^-2
        end
    end
    return he_eos
end

#merge two EOSs (low density and QCD phase)
function mergeEOS(eos_matrix::AbstractMatrix{T}, he_eos, eos, k) where T
    N = k + length(he_eos[:,1])
    for j = 1:N
        if j <= k
            eos_matrix = vcat(eos_matrix, [eos[j,1] eos[j,2] eos[j,3]])
        else
            eos_matrix = vcat(eos_matrix, [he_eos[j-k,1] he_eos[j-k,2] he_eos[j-k,3]])
        end 
    end
    return eos_matrix
end

#BUILDING EOS before Λ transition
function build(eos_matrix::AbstractMatrix{T}, eos, matr_ρ, matr_cs, ds) where T
    #find quantities at transition threshold
    k = findpos(eos,ρt,3)
    
    #Evaluating derivative functions to find quantities at threshold
    p1 = derThr(eos, k)
    
    #creating an interpolated function for speed of sound modification
    cs_interp = csInterp(eos, k, p1, matr_ρ, matr_cs, ds)
    
    #define matrix to store new part of EOS
    he_eos = Matrix{Float64}(undef,0,3)
    he_eos = heEOS(he_eos, eos, k, cs_interp)
        
    #merge the two EOSs at right threshold
    eos_matrix = mergeEOS(eos_matrix, he_eos, eos, k)
    return eos_matrix
end;

In [ ]:
#define interpolated EOS before and after transition
function eos_interp(x) 
    if x < pc
        ϵ_fluid(x)
    elseif x >= pc
        ϵ_fluid(x + Λ) + Λ
    end
end 
#define derivative of interpolated EOS before and after transition
function eos_prime_interp(x) 
    if x < pc
        ϵ_prime(x)
    elseif x >= pc
        ϵ_prime(x + Λ)
    end
end;

In [ ]:
#Computation of mass density, it is needed for moment of inertia calculation
#dln(ρ)=dϵ/(p+ϵ), everything is total=fluid+Λ contribution
function intF(x)
    1/(x - Λ + p_fluid(x - Λ) )
end

#function to define the new ρ after transition
function completeEos(ρ_matr::AbstractMatrix{T}, matr) where T
    #find value of ϵ and ρ just at threshold (outside)
    f(x) = p_fluid(x) - pc
    ϵ_out = find_zero(f, (matr[1,2],matr[end,2]) )
    ρ_out = ρ_fluid(ϵ_out)
    
    #find quantities after the transition at the threshold
    fIn(x) = p_fluid(x) - Λ - pc
    ϵ_fIn = find_zero(fIn, (matr[1,2],matr[end,2]) ) #ϵ_fluid inside at junction
    ϵ_tIn = ϵ_fIn + Λ #ϵ_tot inside at junction
    #we are assuming that ρtot is not only fluid inside, but has Λ contribution
    ρ_In = ρ_out * ((ϵ_tIn + pc)/(ϵ_out + pc))
    
    k = findpos(matr,ϵ_fIn,2) #find closest value to ϵfIn in matrix
     
    ρ_matr = vcat(ρ_matr, [ϵ_tIn ρ_In] )#first element at threshold
    for i=1:length(matr[:,1])-k
        integral, err = quadgk(intF, ϵ_tIn, matr[i+k,2] + Λ)#value of integral and its error
        ρtemp = ρ_In * exp( integral ) 
        ρ_matr = vcat(ρ_matr, [matr[i+k,2]+Λ ρtemp] )
    end
    return ρ_matr
end;

#define interpolated ρ before and after transition
function ρ_interp(x)
    if x < pc
        ρ_fluid(eos_interp(x) )
    elseif x >= pc
        ρ_inside(eos_interp(x) )
    end
end;

#Compute moment of inertia
function InCalc(sol, scaling_function, Rs)
    #remove knots
    r_array = Interpolations.deduplicate_knots!(sol.t;move_knots = false)
    pressure = linear_interpolation(r_array,sol[3,:])
    h_func = linear_interpolation(sol.t,sol[2,:])
    gtt(x) = scaling_function * h_func(x)
    grrInv = linear_interpolation(sol.t,sol[1,:])
    densityI(x) = ρ_interp(pressure(x)) * x^4 * sqrt( gtt(x)/grrInv(x) )
    
    integral, err = quadgk(densityI, r0, Rs)
    return 4.0 * π * integral
end;

In [ ]:
function integrator(P0)
    #condition when defining surface and stop integration
    condition(u, r, integrator) =  u[3]/P0 < 10.0^-12 #10^-10
    affect!(integrator) = terminate!(integrator)
    cb = DiscreteCallback(condition, affect!)
    
    u0 = [f0, h0, P0, H0, β0]
    
    prob = ODEProblem(tov!, u0, rspan)
    
    sol = solve(prob, RK4(), callback = cb, dt = 0.0005, adaptive = false)
    return sol
end;

#Define the system of TOV and tidal deformability eqs to solve
#Note that ϵ'[r]=dϵ/dp*p'=dϵ/dp*du[3]
function tov!(du, u, p, r) #u=f,h,P,H,β
    du[1] = (1.0 - u[1] - 8.0 * π * r^2 * eos_interp(u[3]) )/r
    du[2] = -(u[2] * (-1.0 + u[1] - 8.0 * π * r^2 * u[3]))/(r * u[1])
    du[3] = (-1.0 + u[1] - 8.0 * π * r^2 * u[3]) * (u[3] + 
        eos_interp(u[3]) ) /(2.0 * r * u[1])
    du[4] = u[5]
    du[5] = (u[4] * (-u[1]^3 + (1.0 +  8.0 * π * r^2 * u[3])^3 -
            u[1] * (1.0 + 8.0 * π * r^2 * u[3]) * (-3.0 + 60.0 * π * r^2 * u[3] + 
            20.0 * π * r^2 * eos_interp(u[3]) ) 
            + u[1]^2 * (-3.0 + 60.0 * π * r^2 * u[3] +
            8.0 * π * r^3 * eos_prime_interp(u[3]) * (-1.0 + u[1] - 
            8.0 * π * r^2 * u[3]) * (u[3] + eos_interp(u[3]) ) /(2.0 * r * u[1])+
            20.0 * π * r^2 * eos_interp(u[3]) )) + 
            r * u[1] * (-1 + u[1] - 8.0 * π * r^2 * u[3]) * (1.0 + u[1] + 
            4.0 * π * r^2 * u[3] -
            4.0 * π * r^2 * eos_interp(u[3]) ) * u[5])/(r^2 * u[1]^2 * (1.0 - 
            u[1] + 8.0 * π * r^2 * u[3]))
end;

#Solve a cycle of TOV to retrieve a M-R curve (and other relations)
function cycleTOV(DATA_matrix::AbstractMatrix{T}, P0, Pf) where T
    #determine how long the loop is, based on Pf
    if Pf > 10.0^-2
        N = div((10.0^-4 - P0),(2.5*10.0^-6)) + div((10.0^-3 - 10.0^-4),(2.5*10.0^-5)) +
        div((10.0^-2 - 10.0^-3),(2.5*10.0^-4))
    elseif Pf > 10.0^-3
        N = div((10.0^-4 - P0),(2.5*10.0^-6)) + div((10.0^-3 - 10.0^-4),(2.5*10.0^-5)) + 
        div((Pf - 10.0^-3),(2.5*10.0^-4))
    elseif Pf > 10.0^-4
        N = div((10.0^-4 - P0),(2.5*10.0^-6)) + div((Pf - 10.0^-4),(2.5*10.0^-5))
    elseif Pf > 10.0^-5
        N = div((Pf - P0),(2.5*10.0^-6))
    end

    for i = 1:N
        sol = integrator(P0)
    
        Rs = sol.t[end] #radius
        M = Rs/2 * (1 - sol[1,end]) #mass
        scaling_function = (1 - 2 * M / Rs) / sol[2,end] #rescale for h
        y = Rs * sol[5,end]/sol[4,end]
        C = M / (Rs / Lu * 10^-5) #compactness
        #Love number k2
        k2 = (8.0 * (1.0 - 2.0 * C)^2 * C^5 * (2.0 + 2.0 * C * (-1.0 + y) -
            y))/(5.0 * (2.0 * C * (6.0 + C^2 * (26.0 - 22.0 * y) - 3.0 * y +
            4 * C^4 * (1.0 + y) + 3.0 * C * (-8.0 + 5.0 * y) + C^3 * (-4.0 + 6.0 * y)) +
            3.0 * (1.0 - 2.0 * C)^2 * (2.0 + 2.0 * C * (-1.0 + y) - y) * log(1 - 2*C)))
        λ = 2.0/3.0 * k2 * (Rs / Lu * 10^-5)^5 #tidal deformability
        
        #compute inertia if required
        if Inertia == 1
            I = InCalc(sol, scaling_function, Rs)
        else
            I = 0
        end
        
        #store values in matrix, create a new row each M-R loop
        DATA_matrix = vcat(DATA_matrix, [P0/ρu M Rs/Lu*10.0^-5 C k2 λ I/M^3 λ/M^5])
    
        #increment on P0
        if P0 < 10.0^-4       
            P0 = P0 + 2.5*10.0^-6
        elseif P0 < 10.0^-3
            P0 = P0 + 2.5*10.0^-5
        elseif P0 < 10.0^-2
            P0 = P0 + 2.5*10.0^-4
        else 
            P0 = P0 + 2.5*10.0^-3
        end
    end
    return DATA_matrix
end;

In [ ]:
#Computation of radius and tidal deformability and fixed mass
function λRCalc(DATA_matrix, iMax)
    #remove knots
    M_array = Interpolations.deduplicate_knots!(DATA_matrix[1:iMax,2];move_knots = true)
    
    #interpolate R-M curve and find corresponding radius for Mtest
    RMcurve = linear_interpolation(M_array,DATA_matrix[1:iMax,3])
    Rt = RMcurve(Mtest)
    
    #interpolate λ-M curve and find corresponding λ for Mtest
    λMcurve = linear_interpolation(M_array,DATA_matrix[1:iMax,6])
    λt = λMcurve(Mtest) * KK^5/(Mtest)^5 #scaling factor to check with GW results
    
    return λt, Rt
end;

## Integration on entire dataset
Here we solve the TOV equations for the full M-R curve for all EOSs. We save only configurations that survive the maximum mass test. 

We also perform the study on R and λ for a fixed mass, which can be chosen by changing Mtest.

For this part of the integration, there is no need for the moment of inertia analysis.

In [ ]:
const ρt = 2.0 * n0 * m * ρu #mass density when transition happens
const ρfin = 12.0 * n0 * m * ρu
const pc = 200.0^4 * Kp * Pu #threshold corresponds to QCD scale
#define initial values for solving integration as global
const r0 = 10^-5
const a0 = 1.0
const f0 = 1.0
const h0 = 1.0
const H0 = a0 * r0^2
const β0 = 2. * a0 * r0
const rspan = (r0, 200)
#choose mass for tidal def analysis: e.g. 1.4, 2.18
const Mtest = 1.4
#Array of possible values of Λ
Λarray = [-194.0^4 * Kp * Pu, -150.0^4 * Kp * Pu, -120.0^4 * Kp * Pu, -95.0^4 * Kp * Pu,
    -50.0^4 * Kp * Pu, 0., 50.0^4 * Kp * Pu, 95.0^4 * Kp * Pu, 120.0^4 * Kp * Pu,
    194.0^4 * Kp * Pu];

In [ ]:
Λ = 0.
Inertia = 0
ϵ_fluid = zeros(0)
ϵ_prime(x::Any) = zeros(0)
ρ_fluid = zeros(0)
ρ_inside = zeros(0)
p_fluid = zeros(0)
#HERE CHOOSE EOS: SLY OR AP4!!!
eos = ap4 #sly

tag1 = length(matrix_ρ[:,1]) #identify dataset to name file
tag2 = "ap4" #"sly" #Change accordingly!

for q = 1:length(Λarray)
    Λ = Λarray[q]
    @show Λ
    for j = 1:length(matrix_ρ[:,1])
        z = j
        @show j
    
        eos_matrix = Matrix{Float64}(undef,0,3) #define matrix to store complete EOS
        eos_matrix = build(eos_matrix, eos, matrix_ρ, matrix_cs, z)
        
        #ϵ[p] before transition
        ϵ_fluid = linear_interpolation(eos_matrix[:,1],eos_matrix[:,2])
        #define interpolated dϵ[p]/dp
        ϵ_prime(x) = only(Interpolations.gradient(ϵ_fluid, x) );
        #pFluid before Λ transition
        p_fluid = linear_interpolation(eos_matrix[:,2],eos_matrix[:,1])
        
        #INTEGRATION
        P0 = 2.5 * 10^-5  #this will vary at each step of loop
        #Find what Pf is
        if Λ > 0 && eos_matrix[end,1] > pc
            Pf = eos_matrix[end,1] - Λ
        else
            Pf = eos_matrix[end,1]
        end
    
        #create empty matrix to store data in loop
        DATA_matrix = Matrix{Float64}(undef,0,8) 

        DATA_matrix = cycleTOV(DATA_matrix, P0, Pf)
        
        Mmax, iMax = findmax(DATA_matrix[:,2])
        
        if Λ == 0.0
            temp = 0.0
        else
            temp = abs(Λ/(Kp * Pu))^(1/4)/sign(Λ)
        end
        
        #The next 3 lines are for initial study of complete dataset - comment if not needed
        dumping = open("grid_$tag2"*"_$tag1.csv", "a");
        writedlm( dumping,  [q z Mmax], ' ')
        close(dumping)
        
        #Store data for 'admissible' EOS
        if 2.18 < Mmax < 2.52
            #Find R and λ at Mtest
            λt, Rt = λRCalc(DATA_matrix, iMax)
            
            #dump data on file
            #Note: λt is rescaled in order to be compared to GWs results
            dumping = open("dataProva_$tag2"*"_$tag1"*"_M$Mtest.csv", "a");
            writedlm( dumping,  [temp z Mmax λt Rt], ' ')
            close(dumping)
        end
    end
end;

## Integration for a single EOS
Here we solve the TOV equations for the full M-R curve (and other relations) for a specific EOS. It is possible to create a double loop on Λ and z (bookkeping parameter to identify a specific QCD modification) to save all 'admissible' configurations.

In [ ]:
const ρt = 2.0 * n0 * m * ρu #mass density when transition happens
const ρfin = 12.0 * n0 * m * ρu
const pc = 200.0^4 * Kp * Pu #threshold corresponds to QCD scale
#define initial values for solving integration as global
const r0 = 10^-5
const a0 = 1.0
const f0 = 1.0
const h0 = 1.0
const H0 = a0 * r0^2
const β0 = 2. * a0 * r0
const rspan = (r0, 200)
#Array of possible values of Λ
Λarray = [-194.0^4 * Kp * Pu, -150.0^4 * Kp * Pu, -120.0^4 * Kp * Pu, -95.0^4 * Kp * Pu,
    -50.0^4 * Kp * Pu, 0., 50.0^4 * Kp * Pu, 95.0^4 * Kp * Pu, 120.0^4 * Kp * Pu,
    194.0^4 * Kp * Pu];

In [ ]:
#Choose if including also moment of inertia analysis: 0=no, 1=yes
Inertia = 1 #0

ϵ_fluid = zeros(0)
ϵ_prime(x::Any) = zeros(0)
ρ_fluid = zeros(0)
ρ_inside = zeros(0)
p_fluid = zeros(0)
#HERE CHOOSE EOS: SLY OR AP4, Λ and z
eos = ap4 #sly
Λ = Λarray[2]
z = 45

tag1 = length(matrix_ρ[:,1]) #identify dataset to name file
tag2 = "ap4" #"sly" #Change accordingly!
DATA_matrix = Matrix{Float64}(undef,0,8)

#Cycle on Λ, uncomment if needed
#for q = 1:length(Λarray)
#    Λ = Λarray[q]
#    @show Λ

#Cycle on z, uncomment if needed
#for j = 1:length(matrix_ρ[:,1])
#    z = j
 #   @show j

#Define matrix to store data
DATA_matrix = Matrix{Float64}(undef,0,8)

eos_matrix = Matrix{Float64}(undef,0,3) #define matrix to store complete eos
eos_matrix = build(eos_matrix, eos, matrix_ρ, matrix_cs, z)
        
#ϵ[p] before transition
ϵ_fluid = linear_interpolation(eos_matrix[:,1],eos_matrix[:,2])
#define interpolated dϵ[p]/dp
ϵ_prime(x) = only(Interpolations.gradient(ϵ_fluid, x) );
#pFluid before Λ transition
p_fluid = linear_interpolation(eos_matrix[:,2],eos_matrix[:,1]) #p[ϵ]
#ρ[ϵ] before transition
ρ_fluid = linear_interpolation(eos_matrix[:,2],eos_matrix[:,3])

#be careful not to exit table
if eos_matrix[end,1] >= pc + Λ && eos_matrix[end,1] >= pc
    ρ_matrix = Matrix{Float64}(undef,0,2) #create matrix to store new values of ρ
    ρ_matrix = completeEos(ρ_matrix, eos_matrix) #ϵtot, ρtot after Λ jump
    #Called in ρ_interp
    if length(ρ_matrix[:,1]) > 1
        ρ_inside = linear_interpolation(ρ_matrix[:,1],ρ_matrix[:,2]);
    else
        #remote case where only last element of matrix should be changed
        #I ignore this case
        ρ_inside = ρ_fluid 
    end
end
        
#INTEGRATION
P0 = 2.5 * 10^-5  #this will vary at each step of loop
#Find what Pf is, paying attention not to exit table
#Note that there is a bit of extra carefulness for the case where eos_matrix[end,1] < pc+Λ
#In principle I could go up to threshold for Λ transition, as long as it is not triggered
#I prefer to pick Pf = eos_matrix[end,1] - Λ < pc, and not to risk hitting that threshold
if Λ > 0 && eos_matrix[end,1] > pc 
    Pf = eos_matrix[end,1] - Λ
else
    Pf = eos_matrix[end,1]
end

#Data stored in DATA_matrix: P0/ρu, M, Rs/Lu*10.0^-5, C, k2, λ, I/M^3, λ/M^5
#Note λ is not rescaled with KK^5/M^5
@time DATA_matrix = cycleTOV(DATA_matrix, P0, Pf)

Mmax, iMax = findmax(DATA_matrix[:,2])
        
if Λ == 0.0
    temp = 0.0
else
    temp = abs(Λ/(Kp * Pu))^(1/4)/sign(Λ)
end
        
#Store data for 'admissible' EOS
if 2.18 < Mmax < 2.52
    #dump data on file - comment if not required
    dumping = open("TOV_$tag2"*"_$temp"*"_$z.csv", "a");
    writedlm( dumping,  DATA_matrix, ' ')
    close(dumping)
end;
#end; #Cycle on z, uncomment if needed
#end; #Cycle on Λ, uncomment if needed

In [ ]:
#Example plot: M-R curve
plot(DATA_matrix[:,3],DATA_matrix[:,2], xlabel="R", ylabel="M")

## Combined tidal deformability
Here we compute the combined tidal deformability using the λ-M curves obtained in the previous section for a specific modified EOS.

In [ ]:
#Choose the specific modified EOS defined by ap4/sly, Λ and z.
Λ = -150.0
z = 45
eos = "ap4" #"sly"
TOVdata = readdlm("TOV_$eos"*"_$Λ"*"_$z.csv")

#Choose chirp mass e.g. 1.65, 1.188
Chirp = 1.65
_, k = findmax(TOVdata[:,2])

#We scan only the stable branch of the λ/M^5-M curve
for i = 1:k
    #Note that the KK^5 factor rescales the tidal deformability
    m1, λ1 = TOVdata[k+1-i,2], TOVdata[k+1-i,8] * KK^5
      
    f(x) = (x * m1)^(3/5)/(x + m1)^(1/5) - Chirp
    m2 = find_zero(f, (TOVdata[1,2],TOVdata[end,2]) )
    
    #Find λ for the companion
    m_array = Interpolations.deduplicate_knots!(TOVdata[:,2];move_knots = true)
    λ_of_m = linear_interpolation(m_array,TOVdata[:,8])
    λ2 = λ_of_m(m2) * KK^5 #Note that the KK^5 factor rescales the tidal deformability
       
    if m2 >= m1
        break
    end
    
    #compute combined tidal deformability
    λc = (16.0/13.) * ((m1 + 12. * m2) * m1^4 * λ1 +
        (m2 + 12. * m1) * m2^4 * λ2) / (m1 + m2)^5
    
    #save data for the specific EOS
    dumping = open("combTid_$eos"*"_$Λ"*"_$z"*"_$Chirp.csv", "a")
    writedlm( dumping,  [m1 m2 λc Chirp], ' ')
    close(dumping)
end

In [ ]:
#Example of plot
Λ = -150.0
z = 45
eos = "ap4" #"sly"
Chirp = 1.65
data = readdlm("combTid_$eos"*"_$Λ"*"_$z"*"_$Chirp.csv")
plot(data[:,1], data[:,3], xlabel="M_1", ylabel="λc")